In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,models,layers

In [3]:
import PIL
import cv2
import os

In [4]:
import pathlib
data_dir=pathlib.Path('dataset/train')
data_dir

WindowsPath('dataset/train')

In [5]:
fruit_image={
     'fresh':list(data_dir.glob('fresh/*')),
     'rotten':list(data_dir.glob('rotten/*'))
 }

In [6]:
fruit_image['fresh'][1]

WindowsPath('dataset/train/fresh/rotated_by_15_Screen Shot 2018-06-08 at 4.59.44 PM.png')

In [7]:
fruit_label={
    'fresh':0,
    'rotten':1,
}

In [8]:
img=cv2.imread(str(fruit_image['fresh'][1]))
img

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [9]:
x,y=[],[]
for fruit_quality,images in fruit_image.items():
    for image in images:
        img=cv2.imread(str(image))
        x.append(img)
        y.append(fruit_label[fruit_quality])


In [10]:

img_shape = (180, 180)
x_resized = [cv2.resize(img, img_shape) for img in x]
x = np.array(x_resized)

In [14]:
y= np.array(y)

In [15]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
x_train.shape

(8720, 180, 180, 3)

In [17]:
y_train.shape

(8720,)

In [19]:
x_train= x_train / 255.0
x_test= x_test / 255.0

In [20]:
x_train[0]

array([[[0.94117647, 0.9372549 , 0.94509804],
        [0.94117647, 0.9372549 , 0.94509804],
        [0.94117647, 0.9372549 , 0.94509804],
        ...,
        [0.96862745, 0.96862745, 0.96862745],
        [0.96862745, 0.96862745, 0.96862745],
        [0.96862745, 0.96862745, 0.96862745]],

       [[0.94117647, 0.9372549 , 0.94509804],
        [0.94117647, 0.9372549 , 0.94509804],
        [0.94117647, 0.9372549 , 0.94509804],
        ...,
        [0.96862745, 0.96862745, 0.96862745],
        [0.96862745, 0.96862745, 0.96862745],
        [0.96862745, 0.96862745, 0.96862745]],

       [[0.94117647, 0.9372549 , 0.94509804],
        [0.94117647, 0.9372549 , 0.94509804],
        [0.94117647, 0.9372549 , 0.94509804],
        ...,
        [0.96862745, 0.96862745, 0.96862745],
        [0.96862745, 0.96862745, 0.96862745],
        [0.96862745, 0.96862745, 0.96862745]],

       ...,

       [[0.91372549, 0.92941176, 0.9372549 ],
        [0.91372549, 0.92941176, 0.9372549 ],
        [0.91372549, 0

In [23]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(180, 180, 3)),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.3)
])

e:\fruits-fresh-rotten-classification\venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
fruit_model=models.Sequential([
    data_augmentation,
    layers.Conv2D(32,kernel_size=(3,3),activation='relu',strides=(1,1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3),strides=(1,1), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Conv2D(128, kernel_size=(3, 3),strides=(1,1), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2, activation='softmax' )
])

fruit_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

fruit_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 27, 27, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,327,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,428,866 (5.45 MB)

 Trainable params: 1,428,866 (5.45 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
fruit_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 231s 760ms/step - accuracy: 0.7421 - loss: 0.4803 - val_accuracy: 0.8932 - val_loss: 0.2847
Epoch 2/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 148s 538ms/step - accuracy: 0.9173 - loss: 0.2225 - val_accuracy: 0.9303 - val_loss: 0.1742
Epoch 3/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 155s 566ms/step - accuracy: 0.9271 - loss: 0.1917 - val_accuracy: 0.8716 - val_loss: 0.2779
Epoch 4/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 146s 535ms/step - accuracy: 0.9301 - loss: 0.1791 - val_accuracy: 0.9482 - val_loss: 0.1267
Epoch 5/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 145s 529ms/step - accuracy: 0.9431 - loss: 0.1447 - val_accuracy: 0.9133 - val_loss: 0.2146
Epoch 6/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 161s 588ms/step - accuracy: 0.9397 - loss: 0.1472 - val_accuracy: 0.9564 - val_loss: 0.1091
Epoch 7/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 146s 535ms/step - accuracy: 0.9480 - loss: 0.1327 - val_accuracy: 0.9574 - val_loss: 0.1152
Epoch 8/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 164s 602ms/step - accuracy: 0.9562 -

In [26]:
fruit_model.evaluate(x_test, y_test)


69/69 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.9285 - loss: 0.1923


[0.19924387335777283, 0.9243466258049011]

In [27]:
fruit_model.save('fruit_quality_model.h5')

In [28]:
fruit_model.save('fruit_quality_model.h5')